In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import shutil

import numpy as np
import pandas as pd

from library import bs
from library import common as cm
from library import regression_aux as raux

In [ ]:
%run setup.py
%load_ext autoreload
%autoreload 2

In [ ]:
print('Loading data for:', FREQ)
print('Permutation flag:', PERMUTE)
print('VIX flag:', VIX)

In [ ]:
# load data must be after setup, because some of parameters are going to be overwritten.
%run Load_Clean_aux.py normal

In [ ]:
regr_dir = res_dir + 'Regression/'
print(f'Save results at:\n{regr_dir}')

In [ ]:
if os.path.exists(regr_dir):
    shutil.rmtree(regr_dir)
os.makedirs(regr_dir)
shutil.copy(f'setup.py', regr_dir) 

In [ ]:
def save_tables(res, dirs):
    if res is not None:
        res['df_coef'].to_csv(f'{dirs}coef.csv')
        res['df_fit_std'].to_csv(f'{dirs}std.csv')

### NO permute

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}No_Hedge/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        zero = np.array([0.]*len(df_tmp))
        zero = pd.Series(zero, index=df_tmp.index)
        pnl_path = f'{sub_regr_dir}pnl/pnl{i}.csv'
        cm.store_pnl(
            df_tmp, zero,
            pnl_path=pnl_path)

#### BS

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}BS_Benchmark/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        pnl_path = f'{sub_regr_dir}pnl/pnl{i}.csv'
        cm.store_pnl(
            df_tmp, df_tmp['delta_bs'],
            pnl_path=pnl_path)

#### Delta-Vega-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Vanna/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'vega_n', 'vanna_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vega-Gamma-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Gamma_Vanna/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'vega_n', 'gamma_n', 'vanna_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

### Permute

#### Prepare all permuted data sets

In [ ]:
if PERMUTE:
    train_permutes, test_permutes = [], []
    for i in range(NUM_TEST):
        # the union of train and test
        df_permute = df_train.append(mc_sets[i], ignore_index=True, sort=False)
        df_permute['Is_In_Some_Test'] = False
        df_permute = cm.permute_core(df_permute, 0, random_seed=i)

        df_train_permuted = df_permute.loc[((df_permute['period0'] == 0) | (df_permute['period0'] == 1))]
        df_test_permuted = df_permute.loc[df_permute['period0'] == 2]

        train_permutes.append(df_train_permuted.copy())
        test_permutes.append(df_test_permuted.copy())

#### No hedge

In [ ]:
if PERMUTE:
    print('Permuted!')
    sub_res = f'{regr_dir}No_Hedge/pnl/'
    os.makedirs(sub_res, exist_ok=True)
    for i in range(NUM_TEST):
        df_tmp = train_permutes[i].append(test_permutes[i])
        zero = np.array([0.]*len(df_tmp))
        zero = pd.Series(zero, index=df_tmp.index)
        cm.store_pnl(
            df_tmp, zero,
            pnl_path=f'{sub_res}pnl{i}.csv')

#### BS
Permuting affects the Black-Scholes, because the out-of-sample set is different.

In [ ]:
if PERMUTE:
    print('Permuted!')
    sub_res = f'{regr_dir}BS_Benchmark/pnl/'
    os.makedirs(sub_res, exist_ok=True)
    for i in range(NUM_TEST):
        df_tmp = train_permutes[i].append(test_permutes[i])
        cm.store_pnl(
            df_tmp, df_tmp['delta_bs'],
            pnl_path=f'{sub_res}pnl{i}.csv')

#### Delta-Vega-Vanna

In [ ]:
if PERMUTE:
    print('Permuted!')

    for i in range(NUM_TEST):    
        df_tmp = train_permutes[i].append(test_permutes[i])
        kwargs = {'vix': VIX, 
              'features': ['delta_bs', 'vega_n', 'vanna_n'], 
              'max_period': 0, 
              'sub_res': f'{regr_dir}Delta_Vega_Vanna/pnl/',
              'pnl_path': f'{regr_dir}Delta_Vega_Vanna/pnl/pnl{i}.csv',
              'df': df_tmp,
              'delta_coeff_1': False,
              'agg_side': False
         }
        res = raux.run_store_lin(**kwargs)
        print(res['df_coef'])